## Data Cleaning
### Thea Yang, Nick Gammal, Nick Hausman, Charlie Ward

Cleaning file: `application_train.csv`

In [100]:
#importing libraries
import pandas as pd
import numpy as np
from Models import Logistic, Model
PATH_EXTRA = 'home-credit-default-risk/'

In [72]:
# reading in data
# df = pd.read_csv("application_train.csv")
df = pd.read_csv(PATH_EXTRA + "application_train.csv")

In [73]:
"""
dropping columns that either had too high correlation with other columns or 
too many missing obs that could not be imputed or modified

see report for full description on decision to drop these columns
""" 
df = df.drop(columns=['AMT_GOODS_PRICE', 'CNT_CHILDREN', 'FLAG_EMP_PHONE', 'REGION_RATING_CLIENT_W_CITY', 
 'REG_REGION_NOT_WORK_REGION', 'LIVE_CITY_NOT_WORK_CITY', 'LIVINGAPARTMENTS_MEDI', 
 'ELEVATORS_MEDI', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'LIVINGAREA_MEDI', 'EXT_SOURCE_1'])

In [74]:
# dropping obs for rows that have very little missing values or can't be modified and we still want to keep
clean_df = df[df['DAYS_LAST_PHONE_CHANGE'].notnull() & 
   df['CNT_FAM_MEMBERS'].notnull() & 
   df['EXT_SOURCE_2'].notnull() & 
   df['DEF_30_CNT_SOCIAL_CIRCLE'].notnull() &
   df['OBS_30_CNT_SOCIAL_CIRCLE'].notnull() &
   df['EXT_SOURCE_3'].notnull()
  ]

# imputing amt ammunity column with mean amt
mean_amt_annuity = clean_df.loc[:,'AMT_ANNUITY'].mean()
clean_df.fillna({'AMT_ANNUITY':mean_amt_annuity}, inplace=True)

/var/folders/yd/nfpg90257z743pz74p6rft340000gn/T/ipykernel_59311/3801100754.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df.fillna({'AMT_ANNUITY':mean_amt_annuity}, inplace=True)


### Functions

In [75]:
def map_amt_req(c):
    if c >= 1:
        return 1
    else:
        return 0
    
# discretize own car age
def agemap(num):
    if num <= 1:
        return "new"
    elif num <= 5:
        return "young"
    elif num <= 10:
        return "middle"
    elif num <= 20:
        return "aging"
    elif num <= 60:
        return "old"
    elif num > 60:
        return "classic"
    else:
        return "no car"

# Refactor occupation type
blue = ["Laborers", "Drivers", "Medicine staff", "Security staff", "Cooking staff", "Cleaning staff", "Private service staff", "Low-skill Laborers", "Secretaries", "Waiters/barmen staff"]
white = ["Sales staff", "Core staff", "Managers", "High skill tech staff", "Accountants", "Realty agents", "HR staff", "IT staff"]
def workmap(job):
    if job in blue:
        return "blue"
    elif job in white:
        return "white"
    else:
        return "other"
    
def accompany_map(c):
    if c == 'Unaccompanied':
        return 'Unaccompanied'
    elif c in ['Family', 'Spouse, partner', 'Children', 'Other_B', 'Other_A', 'Group of people']:
        return 'Accompanied'
    else:
        return 'Unknown'

In [76]:
# makng new column based on whether the person has made an enquiry to the Credit Bureau at all in the past year
clean_df.loc[:, 'SUM_AMT_REQ_CREDIT'] = clean_df[['AMT_REQ_CREDIT_BUREAU_MON',
'AMT_REQ_CREDIT_BUREAU_WEEK',
'AMT_REQ_CREDIT_BUREAU_DAY',
'AMT_REQ_CREDIT_BUREAU_HOUR',
'AMT_REQ_CREDIT_BUREAU_QRT',
'AMT_REQ_CREDIT_BUREAU_YEAR']].sum(axis=1)

clean_df.loc[:, 'AMT_REQ_CREDIT']  = clean_df['SUM_AMT_REQ_CREDIT'].apply(map_amt_req)

clean_df = clean_df.drop(columns=['AMT_REQ_CREDIT_BUREAU_WEEK',
'AMT_REQ_CREDIT_BUREAU_DAY',
'AMT_REQ_CREDIT_BUREAU_HOUR',
'AMT_REQ_CREDIT_BUREAU_QRT',
'AMT_REQ_CREDIT_BUREAU_YEAR',
'AMT_REQ_CREDIT_BUREAU_MON',
'SUM_AMT_REQ_CREDIT'])

/var/folders/yd/nfpg90257z743pz74p6rft340000gn/T/ipykernel_59311/4116833493.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df.loc[:, 'SUM_AMT_REQ_CREDIT'] = clean_df[['AMT_REQ_CREDIT_BUREAU_MON',
/var/folders/yd/nfpg90257z743pz74p6rft340000gn/T/ipykernel_59311/4116833493.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df.loc[:, 'AMT_REQ_CREDIT']  = clean_df['SUM_AMT_REQ_CREDIT'].apply(map_amt_req)


In [77]:
# remapping categorical columns to simplify levels and account for missing values
clean_df['OCCUPATION_TYPE'] = clean_df['OCCUPATION_TYPE'].apply(workmap)
clean_df['OWN_CAR_AGE'] = clean_df['OWN_CAR_AGE'].apply(agemap)
clean_df['NAME_TYPE_SUITE']= clean_df['NAME_TYPE_SUITE'].apply(accompany_map)

In [78]:
# get all the 'housing-related' columns and keep only the median ones
mode_cols = np.array(clean_df.columns[clean_df.columns.str.contains("_MODE")])
avg_cols = np.array(clean_df.columns[clean_df.columns.str.contains("_AVG")])
med_cols = np.array(clean_df.columns[clean_df.columns.str.contains("_MEDI")])

clean_df = clean_df.drop(columns=mode_cols).drop(columns=avg_cols)

In [79]:
# code related to created a 'HOUSING SCORE' based on number of housing columns that are above average for that row
clean_df = clean_df.reset_index(drop=True)
combine = clean_df[med_cols]

housing_columns_above_mean_cnt = pd.Series(np.zeros(len(combine.index)))
for col in combine.columns:
    housing_columns_above_mean_cnt += (combine[col] > combine[col].mean()).astype(int)

na_bool_series = [combine[col].isna() for col in combine.columns]
undefined_housing_indicies = []
for i in range(len(na_bool_series[0])):
    if all(l[i] for l in na_bool_series):
        undefined_housing_indicies.append(i)
        
for i in undefined_housing_indicies: housing_columns_above_mean_cnt[i] = 'NO INFO' 
    
clean_df['HOUSING_SCORE'] = housing_columns_above_mean_cnt
# dropping the original columns
clean_df = clean_df.drop(columns=med_cols)

In [80]:
# final filter to get columns with only known housing scores
clean_df_2 = clean_df[clean_df['HOUSING_SCORE'] != 'NO INFO']

In [81]:
# check for missing values
test = pd.DataFrame(clean_df.isna().sum()).reset_index()
test.columns = ['name', 'count']
test[test['count'] > 0 ]

,name,count


## Adding Additional Features from Supplemental Tables

In [82]:
# removing flag document columns
clean_df_2 = clean_df_2.drop(clean_df_2.loc[:,'FLAG_DOCUMENT_2':'FLAG_DOCUMENT_21' ].columns, axis=1)

In [83]:
# reading in additional tables
df_credit = pd.read_csv(PATH_EXTRA+'credit_card_balance.csv')
df_prev = pd.read_csv(PATH_EXTRA+'previous_application.csv')

### Average Annuity Credit Ratio from Previous Loans

In [84]:
# get average annuity credit ratio for previous applications
df_prev['ANNUITY_CREDIT_RATIO'] = df_prev['AMT_ANNUITY']/df_prev['AMT_CREDIT']
avg_ann_cred_ratio = pd.DataFrame(df_prev.groupby('SK_ID_CURR')['ANNUITY_CREDIT_RATIO'].mean()).reset_index()

In [85]:
# merge in new feature
df_clean = clean_df_2.merge(avg_ann_cred_ratio, on='SK_ID_CURR', how='left')

In [86]:
df_clean = df_clean[df_clean['ANNUITY_CREDIT_RATIO'].notnull()]

### Number of Months of Missed Minimum Payments

In [87]:
df_credit['payment diff'] = df_credit['AMT_PAYMENT_CURRENT'] - df_credit['AMT_INST_MIN_REGULARITY']
df_credit['CNT_MISSED_MIN'] = np.where(df_credit['payment diff'] < 0, 1, 0)

cnt_missed_min = pd.DataFrame(df_credit.groupby('SK_ID_CURR')['CNT_MISSED_MIN'].sum()).reset_index()

In [88]:
# merge in new feature
df_clean = df_clean.merge(cnt_missed_min, on='SK_ID_CURR', how='left')

In [89]:
# fill in 0 if never missed min
df_clean['CNT_MISSED_MIN'].fillna(0, inplace=True)

### Number of Previous Total Applied Loans & Number of Previous Accepted Loans

In [90]:
# previous total amount of applied loans per id
prev_loan_cnt = pd.DataFrame(df_prev['SK_ID_CURR'].value_counts()).reset_index()
prev_loan_cnt.columns = ['SK_ID_CURR', 'CNT_PREV_LOANS']

In [91]:
# previous total amount of accepted loans per id
prev_acc_loan = pd.DataFrame(df_prev[df_prev['NAME_CONTRACT_STATUS'] == 'Approved']['SK_ID_CURR'].value_counts()).reset_index()
prev_acc_loan.columns = ['SK_ID_CURR', 'CNT_ACCEPTED_LOANS']

In [92]:
# merge in new feature
df_clean = df_clean.merge(prev_loan_cnt, on='SK_ID_CURR', how='left')
df_clean = df_clean.merge(prev_acc_loan, on='SK_ID_CURR', how='left')

In [93]:
# if there is missing data means they never previously applied for loan -> fill in with 0
df_clean['CNT_PREV_LOANS'].fillna(0, inplace=True)
df_clean['CNT_ACCEPTED_LOANS'].fillna(0, inplace=True)

In [94]:
df_clean = df_clean.rename(columns={'ANNUITY_CREDIT_RATIO':'PREV_AVG_AC_RATIO'})

### Bureau data: creating financial bureau score

In [95]:
bb = pd.read_csv(PATH_EXTRA+'bureau_balance.csv')
b = pd.read_csv(PATH_EXTRA+'bureau.csv')

bb = bb.drop('MONTHS_BALANCE', axis=1)
# I dropped months_balance, because it seems mostly useless as were are only looking to penalize late payments, and not consider on time ones. 

b = b.loc[:, ['SK_ID_CURR', 'SK_ID_BUREAU', 'AMT_CREDIT_SUM_OVERDUE', 'AMT_CREDIT_MAX_OVERDUE', 'CREDIT_DAY_OVERDUE', 'AMT_CREDIT_SUM']]
# After redeaing through the bureau.csv descriptions, only this subset seems useful for our calculations.

bb = bb[(bb.STATUS != 'X') & (bb.STATUS != 'C') & (bb.STATUS != '0')] # Only want to see bad examples, not good ones
bb.STATUS = bb.STATUS.astype(int)

comb = pd.merge(b, bb, on='SK_ID_BUREAU', how='left').fillna(0).groupby(['SK_ID_CURR']).sum().drop('SK_ID_BUREAU', axis=1).add_prefix('SUM_OF_')
# Going to replace NA values with 0, seems resonable because other columns are usually 0. Did outer join so we don't eliminate those without a status

df_clean = pd.merge(df_clean, comb, on='SK_ID_CURR')

### Dimensionality reductions

In [96]:
df_clean = df_clean.drop('FLAG_MOBIL', axis=1) # breaks gradient descent

In [97]:
# converting binary column
for col in df_clean.columns:
    if len(df_clean[col].unique()) == 2:
        df_clean[col] = pd.Series(np.where(df_clean[col].values == df_clean[col].unique()[0], 1, 0), df_clean.index)

In [99]:
# export to csv
df_clean.to_csv('cleaned_training_data.csv', index=False)
df_clean.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,NAME_TYPE_SUITE,...,HOUSING_SCORE,PREV_AVG_AC_RATIO,CNT_MISSED_MIN,CNT_PREV_LOANS,CNT_ACCEPTED_LOANS,SUM_OF_AMT_CREDIT_SUM_OVERDUE,SUM_OF_AMT_CREDIT_MAX_OVERDUE,SUM_OF_CREDIT_DAY_OVERDUE,SUM_OF_AMT_CREDIT_SUM,SUM_OF_STATUS
0,100002,1,1,M,1,1,202500.0,406597.5,24700.5,Unaccompanied,...,0.0,0.051670,0.0,1,1.0,0.0,30053.79,0,4343645.565,27.0
1,100016,0,1,F,1,1,67500.0,80865.0,5881.5,Unaccompanied,...,2.0,0.102497,0.0,4,4.0,0.0,0.00,0,474984.000,0.0
2,100017,0,1,M,0,0,225000.0,918468.0,28966.5,Unaccompanied,...,7.0,0.085245,0.0,2,2.0,0.0,0.00,0,859770.000,0.0
3,100022,0,0,F,1,1,112500.0,157500.0,7875.0,Accompanied,...,2.0,0.065560,0.0,1,1.0,0.0,0.00,0,1057500.000,0.0
4,100026,0,1,F,1,0,450000.0,497520.0,32521.5,Unaccompanied,...,3.0,0.117923,0.0,3,2.0,0.0,0.00,0,5625000.000,0.0
